# 第5章: 係り受け解析

日本語Wikipediaの「人工知能」に関する記事からテキスト部分を抜き出したファイルがai.ja.zipに収録されている． この文章をCaboChaやKNP等のツールを利用して係り受け解析を行い，その結果をai.ja.txt.parsedというファイルに保存せよ．このファイルを読み込み，以下の問に対応するプログラムを実装せよ．

In [10]:
!cabocha -f1 -o data/ai.ja/ai.ja.txt.parsed data/ai.ja/ai.ja.txt

## 40. 係り受け解析結果の読み込み（形態素）Permalink
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，係り受け解析の結果（ai.ja.txt.parsed）を読み込み，各文をMorphオブジェクトのリストとして表現し，冒頭の説明文の形態素列を表示せよ．

In [34]:
class Morph():
    def __init__(self,surface,base,pos,pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1
    
    def get_items(self) -> list:
        return {"surface" : self.surface,
               "base" : self.base,
               "pos" : self.pos,
               "pos1" : self.pos1}
def mk_morph(line):
    items = line.split("\t")
    pos = items[1].replace("\n","").split(",")
    return Morph(items[0],pos[6],pos[0],pos[1])

result = []
with open("./data/ai.ja/ai.ja.txt.parsed","r") as f:
    for line in f.readlines():
        if line[0] == "*":
            continue
        elif "EOS" in line:
            continue
        result.append(mk_morph(line))
        
for morph in result[:10]:
    print(morph.get_items())


{'surface': '人工', 'base': '人工', 'pos': '名詞', 'pos1': '一般'}
{'surface': '知能', 'base': '知能', 'pos': '名詞', 'pos1': '一般'}
{'surface': '人工', 'base': '人工', 'pos': '名詞', 'pos1': '一般'}
{'surface': '知能', 'base': '知能', 'pos': '名詞', 'pos1': '一般'}
{'surface': '（', 'base': '（', 'pos': '記号', 'pos1': '括弧開'}
{'surface': 'じん', 'base': 'じん', 'pos': '名詞', 'pos1': '一般'}
{'surface': 'こうち', 'base': 'こうち', 'pos': '名詞', 'pos1': '一般'}
{'surface': 'のう', 'base': 'のう', 'pos': '助詞', 'pos1': '終助詞'}
{'surface': '、', 'base': '、', 'pos': '記号', 'pos1': '読点'}
{'surface': '、', 'base': '、', 'pos': '記号', 'pos1': '読点'}


## 41. 係り受け解析結果の読み込み（文節・係り受け）Permalink
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストの係り受け解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，冒頭の説明文の文節の文字列と係り先を表示せよ．本章の残りの問題では，ここで作ったプログラムを活用せよ．



In [114]:
class Chunk():
    def __init__(self,morphs,dst,srcs):
        self.morphs = morphs
        self.dst = dst
        self.srcs = srcs
    
    def get_morphs(self):
        return "".join([morph.surface for morph in self.morphs])
    
from collections import defaultdict
morphs = []
tmp_morphs = []
chunks = []
dst = -1
srcs = defaultdict(list)
chunk_id = -1
with open("./data/ai.ja/ai.ja.txt.parsed","r") as f:
    for line in f.readlines():
        if line[0] == "*":
            words = line.split(" ")
            if len(tmp_morphs) != 0:
                chunks.append(Chunk(tmp_morphs,dst,srcs[chunk_id]))
            chunk_id = int(words[1])
            tmp_morphs = []
            dst = int(words[2].replace("D",""))
            srcs[dst].append(int(chunk_id))
            continue
        elif "EOS" in line:
            srcs = defaultdict(list)
            continue
        morph = mk_morph(line)
        morphs.append(morph)
        tmp_morphs.append(morph)

In [115]:
for chunk in chunks[:10]:
    print(chunk.get_morphs())
    print(chunk.dst)
    print(chunk.srcs)

人工知能
-1
[]
人工知能
17
[]
（じんこうちのう、、
17
[]
AI
3
[]
〈エーアイ〉）とは、
17
[2]
「『計算
5
[]
（）』という
9
[4]
概念と
9
[]
『コンピュータ
8
[]
（）』という
9
[7]


## 42. 係り元と係り先の文節の表示Permalink
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ

In [113]:
def non_symbol_surface(chunk):
    new_chunk = []
    for token in chunk:
        if token.pos != "記号":
            new_chunk.append(token.surface)
    return "".join(new_chunk)

from cabocha.analyzer import CaboChaAnalyzer
analyzer = CaboChaAnalyzer()

with open("./data/ai.ja/ai.ja.txt","r") as f:
    for line in f.readlines():
        tree = analyzer.parse(line)
        chunks = tree.chunks
        if len(chunks) == 0:
            continue
            
        for chunk in chunks:
            try:
                print(f"{non_symbol_surface(chunk)}\t{non_symbol_surface(chunk.next_link)}")
            except Exception:
                continue



人工知能	語
じんこうちのう	語
AI	エーアイとは
エーアイとは	語
計算	という
という	道具を
概念と	道具を
コンピュータ	という
という	道具を
道具を	用いて
用いて	研究する
知能を	研究する
研究する	計算機科学
計算機科学	の
の	一分野を
一分野を	指す
指す	語
語	研究分野とも
言語の	推論
理解や	推論
推論	問題解決などの
問題解決などの	知的行動を
知的行動を	代わって
人間に	代わって
代わって	行わせる
コンピューターに	行わせる
行わせる	技術または
技術または	研究分野とも
計算機	コンピュータによる
コンピュータによる	情報処理システムの
知的な	情報処理システムの
情報処理システムの	実現に関する
設計や	実現に関する
実現に関する	研究分野とも
研究分野とも	される
日本大百科全書(ニッポニカ)』の	解説で
解説で	述べている
情報工学者通信工学者の	佐藤理史は
佐藤理史は	述べている
次のように	述べている
人間の	知的能力を
知的能力を	実現する
コンピュータ上で	実現する
実現する	技術ソフトウェアコンピュータシステム
様々な	技術ソフトウェアコンピュータシステム
技術ソフトウェアコンピュータシステム	ある
応用例は	ある
自然言語処理	機械翻訳かな漢字変換構文解析等
機械翻訳かな漢字変換構文解析等	専門家の
専門家の	推論判断を
推論判断を	模倣する
模倣する	エキスパートシステム
エキスパートシステム	画像認識等が
画像データを	解析して
解析して	検出抽出したりする
特定の	パターンを
パターンを	検出抽出したりする
検出抽出したりする	画像認識等が
画像認識等が	ある
ある	命名された
1956年に	命名された
ダートマス会議で	命名された
ジョンマッカーシーにより	命名された
命名された	使われている
現在では	使われている
記号処理を	用いた
用いた	知能の
知能の	記述を
記述を	する
主体と	する
する	研究での
情報処理や	研究での
研究での	アプローチという
アプローチという	意味あいでも
意味あいでも	使われている
使われている	ある
家庭用電気機械器具の	制御システムや
制御システムや	思考ルーチンも
ゲームソフトの	思考ルーチンも
思考ルーチンも	呼ばれる
こう	呼ばれる
呼ばれる	ことも


## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出Permalink
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．



In [116]:
def non_symbol_surface(chunk):
    new_chunk = []
    for token in chunk:
        if token.pos != "記号":
            new_chunk.append(token.surface)
    return "".join(new_chunk)

def include_pos(chunk,pos):
    for token in chunk:
        if token.pos == pos:
            return True
    return False

from cabocha.analyzer import CaboChaAnalyzer
analyzer = CaboChaAnalyzer()

with open("./data/ai.ja/ai.ja.txt","r") as f:
    for line in f.readlines():
        tree = analyzer.parse(line)
        chunks = tree.chunks
        if len(chunks) == 0:
            continue
            
        for chunk in chunks:
            try:
                if include_pos(chunk=chunk,pos = "名詞") & include_pos(chunk=chunk.next_link,pos = "動詞"):
                    print(f"{non_symbol_surface(chunk)}\t{non_symbol_surface(chunk.next_link)}")
            except Exception:
                continue



道具を	用いて
知能を	研究する
一分野を	指す
知的行動を	代わって
人間に	代わって
コンピューターに	行わせる
研究分野とも	される
解説で	述べている
佐藤理史は	述べている
次のように	述べている
知的能力を	実現する
コンピュータ上で	実現する
技術ソフトウェアコンピュータシステム	ある
応用例は	ある
推論判断を	模倣する
画像データを	解析して
解析して	検出抽出したりする
パターンを	検出抽出したりする
画像認識等が	ある
1956年に	命名された
ダートマス会議で	命名された
ジョンマッカーシーにより	命名された
命名された	使われている
現在では	使われている
記号処理を	用いた
記述を	する
主体と	する
意味あいでも	使われている
思考ルーチンも	呼ばれる
ことも	ある
カウンセラーを	模倣した
プログラム	出されるが
人工無脳が	出されるが
引き合いに	出されるが
計算機に	させようという
役割を	させようという
エキスパートシステムと	呼ばれる
実現は	困難視されている
人間が	持つ
暗黙に	持つ
記述が	なり
問題と	なり
利用が	困難視されている
困難視されている	ある
アプローチとしては	知られているが
アプローチも	知られているが
差は	ある
記号的明示性に	ある
その後	集めた
サポートベクターマシンが	集めた
注目を	集めた
経験を	行う
元に	行う
学習を	行う
手法も	ある
宇宙において	実装するという
言葉通り	実装するという
知性を	表現し
機械的に	表現し
表現し	実装するという
登場により	行った
流行を	超えて
社会に	浸透して
浸透して	行った
2017年にかけて	導入した
ディープラーニングを	導入した
プレイヤーも	破り
麻雀では	なった
MicrosoftSuphx(SuperPhoenix)」が	到達するなど
AIとして	到達するなど
十段に	到達するなど
到達するなど	なった
最先端技術と	なった
人工知能は	呼ばれ
機械学習と	呼ばれ
ものが	ある
学習を	繰り返す
ことを	している
基本と	している
システム	関係している
経験に	基づく
手法であり	関係している
ソフトコンピューティングと	関係している
関係している	ある
手法としては	ある
ものが	ある
これらを	統合した
知的システムを	

## 44. 係り受け木の可視化Permalink
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，Graphviz等を用いるとよい．

In [132]:
def non_symbol_surface(chunk):
    new_chunk = []
    for token in chunk:
        if token.pos != "記号":
            new_chunk.append(token.surface)
    return "".join(new_chunk)

def include_pos(chunk,pos):
    for token in chunk:
        if token.pos == pos:
            return True
    return False

from cabocha.analyzer import CaboChaAnalyzer
analyzer = CaboChaAnalyzer()

import graphviz
g = graphviz.Digraph(format='svg', filename='cabocha')

with open("./data/ai.ja/ai.ja.txt","r") as f:
    for line in f.readlines():
        tree = analyzer.parse(line)
        chunks = tree.chunks
        if len(chunks) == 0:
            continue
            
        for chunk in chunks:
            try:
                if include_pos(chunk=chunk,pos = "名詞") & include_pos(chunk=chunk.next_link,pos = "動詞"):
                    node1 = non_symbol_surface(chunk)
                    node2 = non_symbol_surface(chunk.next_link)
                    g.node(node1)
                    g.node(node2)
                    g.edge(node1,node2)
            except Exception:
                continue
g.view()

'cabocha.svg'

## 45. 動詞の格パターンの抽出Permalink
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

動詞を含む文節において，最左の動詞の基本形を述語とする
述語に係る助詞を格とする
述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． この文は「作り出す」という１つの動詞を含み，「作り出す」に係る文節は「ジョン・マッカーシーは」，「会議で」，「用語を」であると解析された場合は，次のような出力になるはずである．

作り出す	で は を
このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

コーパス中で頻出する述語と格パターンの組み合わせ
「行う」「なる」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）


In [158]:
def non_symbol_surface(chunk):
    new_chunk = []
    for token in chunk:
        if token.pos != "記号":
            new_chunk.append(token.surface)
    return "".join(new_chunk)

def include_pos(chunk,pos):
    for token in chunk:
        if token.pos == pos:
            return True
    return False

from cabocha.analyzer import CaboChaAnalyzer
analyzer = CaboChaAnalyzer()

result = ""
with open("./data/ai.ja/ai.ja.txt","r") as f:
    for line in f.readlines():
        tree = analyzer.parse(line)
        chunks = tree.chunks
        if len(chunks) == 0:
            continue
        for chunk in chunks:
            if not include_pos(chunk = chunk,pos = "動詞"):
                continue
            for token in chunk.tokens:
                if token.pos == "動詞":
                    base = token.genkei
                    break
            corpus = base
            for prev_chunk in chunk.prev_links:
                for prev_token in prev_chunk.tokens:
                    if prev_token.pos == "助詞":
                        corpus += f"\t{prev_token.surface}"
            result += f"{corpus}\n"
            
with open("./data/ai.ja/ai.ja.txt.corpus","w") as f:
    f.write(result)

## 46. 動詞の格フレーム情報の抽出Permalink
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる
「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． この文は「作り出す」という１つの動詞を含み，「作り出す」に係る文節は「ジョン・マッカーシーは」，「会議で」，「用語を」であると解析された場合は，次のような出力になるはずである．

作り出す	で は を	会議で ジョンマッカーシーは 用語を

In [162]:
def non_symbol_surface(chunk):
    new_chunk = []
    for token in chunk:
        if token.pos != "記号":
            new_chunk.append(token.surface)
    return "".join(new_chunk)

def include_pos(chunk,pos):
    for token in chunk:
        if token.pos == pos:
            return True
    return False

from cabocha.analyzer import CaboChaAnalyzer
analyzer = CaboChaAnalyzer()

result = ""
with open("./data/ai.ja/ai.ja.txt","r") as f:
    for line in f.readlines():
        tree = analyzer.parse(line)
        chunks = tree.chunks
        if len(chunks) == 0:
            continue
        for chunk in chunks:
            if not include_pos(chunk = chunk,pos = "動詞"):
                continue
            for token in chunk.tokens:
                if token.pos == "動詞":
                    base = token.genkei
                    break
            corpus = base
            for prev_chunk in chunk.prev_links:
                for prev_token in prev_chunk.tokens:
                    if prev_token.pos == "助詞":
                        corpus += f"\t{prev_token.surface}"
            chunks_surface = "\t".join([tmp_chunk.surface for tmp_chunk in chunk.prev_links])
            result += f"{corpus}\t{chunks_surface}\n"
            
with open("./data/ai.ja/ai.ja.txt.corpus","w") as f:
    f.write(result)

## 47. 機能動詞構文のマイニングPermalink
動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．

「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）
例えば「また、自らの経験を元に学習を行う強化学習という手法もある。」という文から，以下の出力が得られるはずである．

学習を行う	に を	元に 経験を

In [188]:
def non_symbol_surface(chunk):
    new_chunk = []
    for token in chunk:
        if token.pos != "記号":
            new_chunk.append(token.surface)
    return "".join(new_chunk)

def include_pos(chunk,pos):
    for token in chunk:
        if token.pos == pos:
            return True
    return False

def include_pos1(chunk,pos1):
    for token in chunk:
        if token.pos1 == pos1:
            return True
    return False   

def include_surface(chunk,surface):
    for token in chunk:
        if token.surface == surface:
            return True
    return False   

from cabocha.analyzer import CaboChaAnalyzer
analyzer = CaboChaAnalyzer()

result = ""
with open("./data/ai.ja/ai.ja.txt","r") as f:
    for line in f.readlines():
        tree = analyzer.parse(line)
        chunks = tree.chunks
        if len(chunks) == 0:
            continue
        for chunk in chunks:
            if not include_pos(chunk = chunk,pos = "名詞"):
                continue
            sahen = False
            if include_pos1(chunk,pos1="サ変接続") and include_surface(chunk,surface="を"):
                try:
                    next_chunk = chunk.next_link
                except:
                    continue
                if not include_pos(next_chunk,"動詞"):
                    continue
                for token in next_chunk.tokens:
                    if token.pos == "動詞":
                        base = token.genkei
                        break

                corpus = chunk.surface + base
                for prev_chunk in chunk.prev_links:
                    for prev_token in prev_chunk.tokens:
                        if prev_token.pos == "助詞":
                            corpus += f"\t{prev_token.surface}"
                chunks_surface = "\t".join([tmp_chunk.surface for tmp_chunk in chunk.prev_links])
                corpus = f"{corpus}\t{chunks_surface}"
                print(corpus)
#with open("./data/ai.ja/ai.ja.txt.corpus","w") as f:
#    f.write(result)

知的行動を代わる	など	の	問題解決などの
推論・判断をする	の	専門家の
記号処理を用いる	
記述をする	の	知能の
注目を集める	
経験を行う	の	自らの
学習を行う	
流行を超える	の	一過性の
学習を繰り返す	や	開発や
推論ルールを、する	の	エキスパートの
統計的学習をする	
進化を見せる	大きな
コンテンツ生成を行う	という	CreativeAIという
機械式計算機をする	の	最初の
開発を行う	の	機械式計算機の
プログラミング言語をする	
テストをする	に関する	知的ふるまいに関する
来談者中心療法を行う	
プログラミング言語をする	
バックギャモン専用コンピュータ・TDギャモンをする	匹敵する
投資全額を上回る	の	政府の
意味付けをする	適した
処理を行う	
知的処理を行う	
意味をする	
知的処理を行う	
研究を進める	の	第五世代コンピュータの
命令をする	の	Prologの
運転をする	
特許をする	の	1/5の
研究をする	について	の	ファジィ制御についての
運転をする	
注目を集める	の	世間の	大きな
ニューロファジィ制御をする	自動化した
成功を受ける	の	試みの
知的制御を用いる	の	同様の
開発工数を抑える	
制御をする	や	の	工作機械や	家電製品の
知的制御をする	
表現する）をする	として	オントロジー	（ファジィルールとして
進歩を担う	の	科学技術の
精度改善を果たす	から	の	従来手法からの	大幅な
専用プログラムを使う	にあたる	解析にあたる
研究を続ける	
行動型システムを用いる	学習する
関連性を導き出す	の	情報の
「ワンショット学習」をする	の	不要の
認識能力を持つ	の	人間並みの
記号接地問題(シンボルグラウンディング問題)をする	
注目を集める	の	識者の
普及を受ける	と	発明と	急速な
機械学習を組み合わせる	と	構築された	神経科学と
投資を行う	莫大な
探索を行う	なき
推論をする	の	ニューラルネットワークの
共同研究を始める	
研究を行う	という	再構成するという
研究開発をする	の	AIの
実験をする	や	の	研究や	新技術の
研究開発をする	の	スーパーコンピュータの
投資をする	
深層学習をする	
脳シミュレーションを行う	
反乱を起こす	
弾圧を併せ持つ	と	の	資金力と	人権

## 48. 名詞から根へのパスの抽出Permalink
文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．

各文節は（表層形の）形態素列で表現する
パスの開始文節から終了文節に至るまで，各文節の表現を” -> “で連結する
「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． CaboChaを係り受け解析に用いた場合，次のような出力が得られると思われる．

ジョンマッカーシーは -> 作り出した
AIに関する -> 最初の -> 会議で -> 作り出した
最初の -> 会議で -> 作り出した
会議で -> 作り出した
人工知能という -> 用語を -> 作り出した
用語を -> 作り出した
KNPを係り受け解析に用いた場合，次のような出力が得られると思われる．

ジョンマッカーシーは -> 作り出した
ＡＩに -> 関する -> 会議で -> 作り出した
会議で -> 作り出した
人工知能と -> いう -> 用語を -> 作り出した
用語を -> 作り出した

In [195]:
def non_symbol_surface(chunk):
    new_chunk = []
    for token in chunk:
        if token.pos != "記号":
            new_chunk.append(token.surface)
    return "".join(new_chunk)

def include_pos(chunk,pos):
    for token in chunk:
        if token.pos == pos:
            return True
    return False


from cabocha.analyzer import CaboChaAnalyzer
analyzer = CaboChaAnalyzer()

result = ""
with open("./data/ai.ja/ai.ja.txt","r") as f:
    for line in f.readlines():
        tree = analyzer.parse(line)
        chunks = tree.chunks
        if len(chunks) == 0:
            continue
        for chunk in chunks:
            if not include_pos(chunk = chunk,pos = "名詞"):
                continue
            corpus = non_symbol_surface(chunk)
            next_chunk = chunk
            while True:
                try:
                    next_chunk = next_chunk.next_link
                except Exception:
                    break
                corpus += f" -> {non_symbol_surface(next_chunk)}"
            print(corpus)


人工知能
人工知能 -> 語 -> 研究分野とも -> される
じんこうちのう -> 語 -> 研究分野とも -> される
AI -> エーアイとは -> 語 -> 研究分野とも -> される
エーアイとは -> 語 -> 研究分野とも -> される
計算 -> という -> 道具を -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
概念と -> 道具を -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
コンピュータ -> という -> 道具を -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
道具を -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
知能を -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
研究する -> 計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
計算機科学 -> の -> 一分野を -> 指す -> 語 -> 研究分野とも -> される
一分野を -> 指す -> 語 -> 研究分野とも -> される
語 -> 研究分野とも -> される
言語の -> 推論 -> 問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
理解や -> 推論 -> 問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
推論 -> 問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
知的行動を -> 代わって -> 行わせる -> 技術または -> 研究分野とも -> される
人間に -> 代わって -> 行わせる -> 技術または 

## 49. 名詞間の係り受けパスの抽出Permalink
文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号がiとj（i<j）のとき，係り受けパスは以下の仕様を満たすものとする．

問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を” -> “で連結して表現する
文節iとjに含まれる名詞句はそれぞれ，XとYに置換する
また，係り受けパスの形状は，以下の2通りが考えられる．

文節iから構文木の根に至る経路上に文節jが存在する場合: 文節iから文節jのパスを表示
上記以外で，文節iと文節jから構文木の根に至る経路上で共通の文節kで交わる場合: 文節iから文節kに至る直前のパスと文節jから文節kに至る直前までのパス，文節kの内容を” | “で連結して表示
「ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。」という例文を考える． CaboChaを係り受け解析に用いた場合，次のような出力が得られると思われる．

Xは | Yに関する -> 最初の -> 会議で | 作り出した
Xは | Yの -> 会議で | 作り出した
Xは | Yで | 作り出した
Xは | Yという -> 用語を | 作り出した
Xは | Yを | 作り出した
Xに関する -> Yの
Xに関する -> 最初の -> Yで
Xに関する -> 最初の -> 会議で | Yという -> 用語を | 作り出した
Xに関する -> 最初の -> 会議で | Yを | 作り出した
Xの -> Yで
Xの -> 会議で | Yという -> 用語を | 作り出した
Xの -> 会議で | Yを | 作り出した
Xで | Yという -> 用語を | 作り出した
Xで | Yを | 作り出した
Xという -> Yを
KNPを係り受け解析に用いた場合，次のような出力が得られると思われる．

Xは | Yに -> 関する -> 会議で | 作り出した。
Xは | Yで | 作り出した。
Xは | Yと -> いう -> 用語を | 作り出した。
Xは | Yを | 作り出した。
Xに -> 関する -> Yで
Xに -> 関する -> 会議で | Yと -> いう -> 用語を | 作り出した。
Xに -> 関する -> 会議で | Yを | 作り出した。
Xで | Yと -> いう -> 用語を | 作り出した。
Xで | Yを | 作り出した。
Xと -> いう -> Yを

In [214]:
def replace_noun(chunk,word):
    new_surface = []
    for token in chunk.tokens:
        if token.pos == "名詞":
            new_surface.append(word)
        else:
            new_surface.append(token.surface)
    return "".join(new_surface)
            
import graphviz
g = graphviz.Digraph(format='svg', filename='cabocha')

text = "ジョンマッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。"
tree = analyzer.parse(text)
chunks = tree.chunks
trees = []
for chunk in chunks:
    if not include_pos(chunk = chunk,pos = "名詞"):
        continue
    next_chunk = chunk
    node1 = replace_noun(next_chunk,"X")
    while True:
        try:
            next_chunk = next_chunk.next_link
        except Exception:
            break

        node2 = next_chunk.surface
        g.node(node1)
        g.node(node2)
        g.edge(node1,node2)
        node1 = node2
    
g.view()

'cabocha.svg'